In [5]:
import os

In [6]:
os.getcwd()

'C:\\Users\\user'

In [7]:
os.chdir("C:\\Users\\user\\Desktop\\SKRIPSI\\SIDANG\Output Final Delta Trimax DFO")

In [8]:
#Importing Modules
import numpy as np
import pandas as pd
import random
import pylab as pyl
from sklearn import preprocessing
from scipy.spatial.distance import hamming

# Inisiasi Nilai Random Untuk Keperluan Simulasi

In [6]:
np.random.uniform()

0.7227536850798388

# Uploading The Data


In [9]:
data = pd.read_excel(r"C:\\Users\\user\\Desktop\\SKRIPSI\\Dataset\\estrogen.xlsx")
#Removing the index
data2 = data.iloc[:,1:]

In [10]:
#Normalisasi Data
data3 = np.copy(data2)
data3 = preprocessing.normalize(data3)

In [11]:
data3 = pd.DataFrame(data3)
data3.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,0.295382,0.294650,0.294822,0.290675,0.287452,0.289812,0.289325,0.286030,0.287616,0.283213,0.282026,0.282677
1,0.282264,0.287873,0.282263,0.275331,0.278789,0.277086,0.285295,0.274063,0.279440,0.319136,0.303948,0.314092
2,0.295907,0.291116,0.289862,0.288586,0.286249,0.288527,0.289301,0.288631,0.284897,0.287478,0.288788,0.284588
3,0.291675,0.291012,0.289909,0.291222,0.283494,0.285030,0.287068,0.288784,0.287491,0.289538,0.291106,0.287643
4,0.286443,0.301306,0.291697,0.291480,0.283896,0.281878,0.290488,0.285096,0.288951,0.294332,0.284885,0.283052


In [12]:
#Converting data to 3D with first dimension being Waktu
t1 = data3.iloc[:,0:3]
t2 = data3.iloc[:,3:6]
t3 = data3.iloc[:,6:9]
t4 = data3.iloc[:,9:12]

In [13]:
D = np.array([t1,t2,t3,t4])
D.shape

(4, 54675, 3)

# Menentukan Delta Untuk Delta Trimax

In [14]:
def msr_calc (D):
    m_iJK = np.nanmean(np.nanmean(D,axis=2),axis=0)
    
    m_iJK = np.expand_dims(np.expand_dims(m_iJK, axis = 0),axis = 2)
    # J kondisi
    m_IjK = np.nanmean(np.nanmean(D,axis=0),axis=0)
    m_IjK = np.expand_dims(np.expand_dims(m_IjK, axis = 0), axis = 0)
    #print(m_IjK)
    #K waktu
    m_IJk = np.nanmean(np.nanmean(D,axis=1),axis=1)
    m_IJk = np.expand_dims(np.expand_dims(m_IJk, axis = 1),axis = 2)
    #print(m_IJk)
    
    m_IJK = np.nanmean(D)
    #print(m_IJK)
    residue = D - m_iJK - m_IjK- m_IJk + (2*m_IJK)
    #print(residue)
    msr = np.mean (np.square(residue))
    return msr

In [15]:
msr_calc(D)

6.719032713403308e-05

# Delta Trimax Algorithm

In [13]:
"""
Created on Sat 03 June 2023
"""

import numpy as np
import sys
import time

class DeltaTrimax():
    def __init__(self,D):
        self.D = D.copy()
        self.D_asli = D.copy()
    
    def hitung_MSR(self, gene, kondisi, waktu, g_add=False, k_add=False, w_add=False):
        
        gene_idx = np.expand_dims(np.expand_dims(np.nonzero(gene)[0],axis=0),axis=2)
        waktu_idx = np.expand_dims(np.expand_dims(np.nonzero(waktu)[0],axis=1),axis=1)
        kondisi_idx = np.expand_dims(np.expand_dims(np.nonzero(kondisi)[0],axis=0),axis=0)
        
        subarr = self.D[waktu_idx, gene_idx, kondisi_idx]
        self.n_gene = subarr.shape[1]
        self.n_kondisi = subarr.shape[2]
        self.n_waktu = subarr.shape[0]
        
        # hitung a_gCT (gene)
        a_gCT = np.nanmean(np.nanmean(subarr,axis=2),axis=0)
        a_gCT = np.expand_dims(np.expand_dims(a_gCT,axis=0),axis=2)
        
        # hitung a_GcT (kondisi)
        a_GcT = np.nanmean(np.nanmean(subarr,axis=0),axis=0)
        a_GcT = np.expand_dims(np.expand_dims(a_GcT,axis=0),axis=1)
        
        # hitung a_GCt (waktu)
        a_GCt = np.nanmean(np.nanmean(subarr,axis=2),axis=1)
        a_GCt = np.expand_dims(np.expand_dims(a_GCt,axis=1),axis=2)
        
        # hitung a_GCT
        a_GCT = np.mean(subarr)
        
        # hitung MSR
        residue = subarr - a_gCT - a_GcT - a_GCt + (2*a_GCT)
        SR = np.square(residue)
        self.MSR = np.mean(SR)
        self.MSR_gene = np.nanmean(np.nanmean(SR,axis=2),axis=0)
        self.MSR_kondisi = np.nanmean(np.nanmean(SR,axis=0),axis=0)
        self.MSR_waktu = np.nanmean(np.nanmean(SR,axis=2),axis=1)
      
        #Node Addition
        if g_add:
            non_gene = np.expand_dims(np.expand_dims(np.nonzero(gene==0)[0],axis=0),axis=2)
            
            
            # hitung a_gCT untuk gene bukan anggota tricluster 
            D_b = self.D.copy()
            D_b[waktu_idx,non_gene, kondisi_idx] = self.D_asli[waktu_idx, non_gene, kondisi_idx]

            subarr_b = D_b[waktu_idx, non_gene, kondisi_idx]

            a_gCT_b = np.nanmean(np.nanmean(subarr_b,axis=2),axis=0)
            a_gCT_b = np.expand_dims(np.expand_dims(a_gCT_b,axis=0),axis=2)
            
            r = subarr_b - a_gCT_b - a_GcT - a_GCt + (2*a_GCT)
            sr_b = np.square(r)
            self.MSR_gene_b = np.nanmean(np.nanmean(sr_b,axis=2),axis=0)
            
        
        if k_add:
            non_kondisi = np.expand_dims(np.expand_dims(np.nonzero(kondisi==0)[0],axis=0),axis=0)
            
            # hitung a_GcT untuk kondisi bukan anggota tricluster
            D_b = self.D.copy()
            D_b[waktu_idx,gene_idx, non_kondisi] = self.D_asli[waktu_idx,gene_idx, non_kondisi]
            subarr_b = D_b[waktu_idx,gene_idx, non_kondisi]

            a_GcT_b = np.nanmean(np.nanmean(subarr_b,axis=0),axis=0)
            a_GcT_b = np.expand_dims(np.expand_dims(a_GcT_b,axis=0),axis=1)
            
            r = subarr_b - a_gCT - a_GcT_b - a_GCt + (2*a_GCT)
            sr_b = np.square(r)
            self.MSR_kondisi_b = np.nanmean(np.nanmean(sr_b,axis=0),axis=0)
        
        if w_add:
            non_waktu = np.expand_dims(np.expand_dims(np.nonzero(waktu==0)[0],axis=1),axis=1)
            
            # hitung a_GCt untuk waktu bukan anggota tricluster
            D_b = self.D.copy()
            D_b[non_waktu, gene_idx, kondisi_idx] = self.D_asli[non_waktu, gene_idx, kondisi_idx]
            subarr_b = D_b[non_waktu, gene_idx, kondisi_idx]

            a_GCt_b = np.nanmean(np.nanmean(subarr_b,axis=2),axis=1)
            a_GCt_b = np.expand_dims(np.expand_dims(a_GCt_b,axis=1),axis=2)

            r = subarr_b - a_gCT - a_GcT - a_GCt_b + (2*a_GCT)
            sr_b = np.square(r)
            self.MSR_waktu_b = np.nanmean(np.nanmean(sr_b,axis=2),axis=1)
         
        
    def multiple_node_deletion(self, gene, kondisi, waktu):
        self.hitung_MSR(gene, kondisi, waktu)
        while (self.MSR > self.delta):
            hapus = 0
            # Hapus gen yang memenuhi kriteria
            if (self.n_gene > self.gene_cutoff):
                gene_dihapus = self.MSR_gene > (self.MSR * self.lamda)
                nonz_idx = gene.nonzero()[0]
                if gene_dihapus.any():
                    hapus = 1
                gene.put(nonz_idx[gene_dihapus],0)
                
                # Hitung kembali MSR
                self.hitung_MSR(gene, kondisi, waktu)
                
            # Hapus kondisi yang memenuhi kriteria
            if (self.n_kondisi > self.kondisi_cutoff):
                kondisi_dihapus = self.MSR_kondisi > (self.MSR * self.lamda)
                nonz_idx = kondisi.nonzero()[0]
                if kondisi_dihapus.any():
                    hapus = 1
                kondisi.put(nonz_idx[kondisi_dihapus],0)
                
                # Hitung kembali MSR
                self.hitung_MSR(gene, kondisi, waktu)
                
            # Hapus waktu yang memenuhi kriteria
            if (self.n_waktu > self.waktu_cutoff):
                waktu_dihapus = self.MSR_waktu > (self.MSR * self.lamda)
                nonz_idx = waktu.nonzero()[0]
                if waktu_dihapus.any():
                    hapus = 1
                waktu.put(nonz_idx[waktu_dihapus],0)
                
                # Hitung kembali MSR
                self.hitung_MSR(gene, kondisi, waktu)
                
            self.muldel += 1
            
            # Stopping Criteria Multiple Node Deletion
            if not hapus:
                break
            sys.stdout.write('\r multiple deletion {} - single deletion {}- node addition {}'.format(
                self.muldel, self.singdel, self.nodeadd
            ))
            sys.stdout.flush()
            
        return gene, kondisi, waktu
  
    def single_node_deletion(self, gene, kondisi, waktu):
        self.hitung_MSR(gene, kondisi, waktu)
        
        while (self.MSR > self.delta):    
            gene_max = np.argmax(self.MSR_gene)
            kondisi_max = np.argmax(self.MSR_kondisi)
            waktu_max = np.argmax(self.MSR_waktu)
            
            if (self.MSR_gene[gene_max] > self.MSR_kondisi[kondisi_max]):
                if (self.MSR_gene[gene_max] > self.MSR_waktu[waktu_max]):
                    # Hapus gen jika memiliki skor residu tertinggi
                    nonz_idx = gene.nonzero()[0]
                    gene.put(nonz_idx[gene_max],0)
                else:
                    # Hapus waktu jika memiliki skor residu tertinggi
                    nonz_idx = waktu.nonzero()[0]
                    waktu.put(nonz_idx[waktu_max],0)
                  
            else:
                if (self.MSR_kondisi[kondisi_max] > self.MSR_waktu[waktu_max]):
                    # Hapus kondisi jika memiliki skor residu tertinggi
                    nonz_idx = kondisi.nonzero()[0]
                    kondisi.put(nonz_idx[kondisi_max],0)
                else:
                    # Hapus Waktu jika memiliki skor residu tertinggi
                    nonz_idx = waktu.nonzero()[0]
                    waktu.put(nonz_idx[waktu_max],0)
    
            self.singdel += 1
            self.hitung_MSR(gene, kondisi, waktu)
            
            sys.stdout.write('\r multiple deletion {} - single deletion {}- node addition {}'.format(
                self.muldel, self.singdel, self.nodeadd
            ))
            sys.stdout.flush()

        return gene, kondisi, waktu
                    
    def node_addition(self, gene, kondisi, waktu):
        self.hitung_MSR(gene, kondisi, waktu)
        while True:
            self.hitung_MSR(gene, kondisi, waktu)
            
            #Penambahan Gen
            self.hitung_MSR(gene, kondisi, waktu, g_add=True)
            no_gene_idx = np.nonzero(gene==0)[0]
            gene_to_add = self.MSR_gene_b < self.MSR
            if gene_to_add.any():
                gene.put(no_gene_idx[gene_to_add],1)
                g_idx = np.expand_dims(np.expand_dims(no_gene_idx[gene_to_add],axis=0),axis=2)
                k_idx = np.expand_dims(np.expand_dims(np.nonzero(kondisi)[0],axis=0),axis=0)
                w_idx = np.expand_dims(np.expand_dims(np.nonzero(waktu)[0],axis=1),axis=2)

                self.D[w_idx, g_idx, k_idx] = self.D_asli[w_idx, g_idx, k_idx]
            # Hitung MSR kembali
            self.hitung_MSR(gene, kondisi, waktu)
            #Penambahan Kondisi
            self.hitung_MSR(gene, kondisi, waktu, k_add =True)
            no_kondisi = np.nonzero(kondisi==0)[0]
            kondisi_to_add = self.MSR_kondisi_b < self.MSR
            if kondisi_to_add.any():
                kondisi.put(no_kondisi[kondisi_to_add],1)

                g_idx = np.expand_dims(np.expand_dims(np.nonzero(gene)[0],axis=0),axis=2)
                k_idx = np.expand_dims(np.expand_dims(no_kondisi[kondisi_to_add],axis=0),axis=0)
                w_idx = np.expand_dims(np.expand_dims(np.nonzero(waktu)[0],axis=1),axis=2)

                self.D[w_idx, g_idx, k_idx] = self.D_asli[w_idx, g_idx, k_idx]
            # Hitung MSR kembali
            self.hitung_MSR(gene, kondisi, waktu)
             
            #Penambahan Waktu
            self.hitung_MSR(gene, kondisi, waktu, w_add=True)
            no_waktu = np.nonzero(waktu==0)[0]
            waktu_to_add = self.MSR_waktu_b < self.MSR
            if waktu_to_add.any():
                waktu.put(no_waktu[waktu_to_add],1)

                g_idx = np.expand_dims(np.expand_dims(np.nonzero(gene)[0],axis=0),axis=2)
                k_idx = np.expand_dims(np.expand_dims(np.nonzero(kondisi)[0],axis=0),axis=0)
                w_idx = np.expand_dims(np.expand_dims(no_waktu[waktu_to_add],axis=1),axis=2)

                self.D[w_idx, g_idx, k_idx] = self.D_asli[w_idx, g_idx, k_idx]
            # Hitung MSR kembali
            self.hitung_MSR(gene, kondisi, waktu)
            self.nodeadd +=1
            if not gene_to_add.any() and not kondisi_to_add.any() and not waktu_to_add.any():
                break
            
            sys.stdout.write('\r multiple deletion {} - single deletion {}- node addition {}'.format(
                self.muldel, self.singdel, self.nodeadd
            ))
            sys.stdout.flush()  
        return gene, kondisi, waktu
    
    
    def mask(self, gene, kondisi, waktu, minval, maxval):
        g = np.expand_dims(np.expand_dims(gene.nonzero()[0],axis=0),axis=2)
        k = np.expand_dims(np.expand_dims(kondisi.nonzero()[0],axis=0),axis=0)
        w = np.expand_dims(np.expand_dims(waktu.nonzero()[0],axis=1),axis=2)
        
        shape = np.count_nonzero(waktu), np.count_nonzero(gene), np.count_nonzero(kondisi)
        if self.mask_mode =='nan':
            mask_val = np.nan
        if self.mask_mode == 'random':
            mask_val = np.random.uniform(self.minval, self.maxval, shape)
        
        self.D[w,g,k] = mask_val
            

    def fit(self, delta, lamda, mask_mode="random", n_triclusters=0):
        '''
        mask_mode : "random", "nan"
        '''
        awal = time.time()
        
        n_waktu, n_gene, n_kondisi = self.D.shape
        
        self.delta = delta
        self.lamda = lamda
        
        # treshold untuk multiple node deletion
        self.gene_cutoff, self.kondisi_cutoff, self.waktu_cutoff = 50,50,50
        #nilai untuk masking
        print(f"mask mode: {mask_mode} \n")
        self.mask_mode = mask_mode
        self.minval = np.min(self.D)
        self.maxval = np.max(self.D)
        
        hasil_gen = []
        hasil_kondisi = []
        hasil_waktu = []
        
        msr = []
        TQI =[]

        i = 1
        
        while True:
            self.muldel = 0
            self.singdel =0
            self.nodeadd = 0

            print("Tricluster ",i)
            waktu = np.ones(n_waktu, dtype=np.int64)
            gene = np.ones(n_gene, dtype=np.int64)
            kondisi = np.ones(n_kondisi, dtype=np.int64)
            
            if self.mask_mode == "nan":
                self.D = self.D.astype(np.float)
                gene = np.mean(np.mean(self.D,axis=0),axis=1)
                gene = np.isnan(gene)==0

            
            # Multiple Node Deletion
            gene, kondisi, waktu = self.multiple_node_deletion(gene, kondisi, waktu)
            
            # Single Node Deletion
            gene, kondisi, waktu = self.single_node_deletion(gene, kondisi, waktu)
            
            # Node Addition
            gene, kondisi, waktu = self.node_addition(gene, kondisi, waktu)
            

            if (gene.sum()==1) or (kondisi.sum()==1) or (waktu.sum()==1):
                akhir = ((time.time()-awal)/60)
                print("\n Waktu Komputasi : {} menit".format(akhir))
                break
            
            
            # Count
            g_count = np.count_nonzero(gene)
            c_count = np.count_nonzero(kondisi)
            t_count = np.count_nonzero(waktu)
            
            # Volume
            volume = g_count * c_count * t_count
            
            # TQI
            tqi = (self.MSR/volume)
            
            print("\n------------- MSR: ",self.MSR)
            print("------------- TQI: ",tqi)
            print('G X S X T:', g_count,' x ',c_count,' x ',t_count)

            
            hasil_gen.append(gene)
            hasil_kondisi.append(kondisi)
            hasil_waktu.append(waktu)

            msr.append(self.MSR)
            TQI.append(tqi)


            if (n_triclusters == i):
                akhir = ((time.time()-awal)/60)
                print("\n Waktu Komputasi : {} menit".format(akhir))
                break
    
    
            # Masking
            self.mask(gene, kondisi, waktu, self.minval, self.maxval)
            
            i+=1
            
        return hasil_gen, hasil_kondisi, hasil_waktu, msr, TQI


# Discrete Firefly Algorithm

In [17]:
from scipy.spatial.distance import hamming
#Firefly Class
class Firefly():
    def __init__(self,tric,D):
        self.tric = tric_awal.copy()
        self.D = D.copy()
        
        
    def hamming_sim (self,i,j):
        x_i = tric_awal.iloc[i,0]+tric_awal.iloc[i,1]+tric_awal.iloc[i,2]
        x_j = tric_awal.iloc[j,0]+tric_awal.iloc[j,1]+tric_awal.iloc[j,2]
        sim = len(x_i) - (hamming(x_i,x_j)*len(x_i))
        r = sim/ len(x_i)
        return r
    
    def mpc (self,i,j,m):
        x_i = tric_awal.iloc[i,0]+tric_awal.iloc[i,1]+tric_awal.iloc[i,2]
        x_j = tric_awal.iloc[j,0]+tric_awal.iloc[j,1]+tric_awal.iloc[j,2]
        nums = list(np.random.choice(54681,m, replace=False))
        for n in nums:
            off1 = x_i[:]
            off2 = x_j[:]
            off1[n:],off2[n:] = off2[n:], off1[n:]
        return off1, off2
    
    def fit_val (self, l):
        fv = np.sum(l[:54675])*np.sum(l[54675:54678])*np.sum(l[54678:])
        return fv
    
    def hitung_delta(self,i):
        if isinstance(i,int):
            genidx = np.expand_dims(np.expand_dims(np.nonzero(tric_awal['gen'][i])[0],axis=0),axis=2)
            konidx = np.expand_dims(np.expand_dims(np.nonzero(tric_awal['kondisi'][i])[0],axis=0),axis=0)
            wakidx = np.expand_dims(np.expand_dims(np.nonzero(tric_awal['waktu'][i])[0],axis=1),axis=1)
        else:
            genidx = np.expand_dims(np.expand_dims(np.nonzero(i[:54675])[0],axis=0),axis=2)
            konidx = np.expand_dims(np.expand_dims(np.nonzero(i[54675:54678])[0],axis=0),axis=0)
            wakidx = np.expand_dims(np.expand_dims(np.nonzero(i[54678:])[0],axis=1),axis=1)


        subarr = D[wakidx, genidx, konidx]
        n_gene = subarr.shape[1]
        n_kondisi = subarr.shape[2]
        n_waktu = subarr.shape[0]

    # hitung m_iJK (gene)
        m_iJK = np.nanmean(np.nanmean(subarr,axis=2),axis=0)
        m_iJK = np.expand_dims(np.expand_dims(m_iJK,axis=0),axis=2)

    # hitung m_IjK (kondisi)
        m_IjK = np.nanmean(np.nanmean(subarr,axis=0),axis=0)
        m_IjK = np.expand_dims(np.expand_dims(m_IjK,axis=0),axis=1)

    # hitung m_IJk (waktu)
        m_IJk = np.nanmean(np.nanmean(subarr,axis=2),axis=1)
        m_IJk = np.expand_dims(np.expand_dims(m_IJk,axis=1),axis=2)

    # hitung m_IJK
        m_IJK = np.mean(subarr)

    # hitung MSR
        residue = subarr - m_iJK - m_IjK - m_IJk + (2*m_IJK)
        SR = np.square(residue)
        MSR = np.mean(SR)
        return MSR

    def fit (self, delta,m,n_itr):
        awal = time.time()
        self.delta = delta
        self.m = m
        self.n_itr = n_itr
        for k in range (n_itr):
            for i in range (0,40):
                for j in range (0,i+1):
                    xi = tric_awal['gen'][i]+tric_awal['kondisi'][i]+tric_awal['waktu'][i]

                    xj = tric_awal['gen'][j]+tric_awal['kondisi'][j]+tric_awal['waktu'][j]

                    if self.fit_val (xi) < self.fit_val (xj):
                        prob = self.hamming_sim (i,j)
                        if random.random() < prob:
                            off1, off2 = self.mpc (i,j,m)
                            if self.fit_val(off1) < self.fit_val(off2):
                                if self.fit_val(xi) < self.fit_val (off2):
                                    if self.hitung_delta(off2) < self.delta:

                                        xi = off2

                            else:
                                if self.fit_val(xi) < self.fit_val(off1):
                                    if self.hitung_delta(off1) < self.delta:

                                        xi=off1
                            tric_awal['gen'][i]=xi[:54675]
                            tric_awal['kondisi'][i]=xi[54675:54678]
                            tric_awal['waktu'][i]=xi[54678:]
            
        print('Waktu Komputasi: ', str((time.time()-awal)/60))
        return tric_awal
        

In [15]:
#Hitung Delta Class
def hitung_delta(tric,i):
    if isinstance(i,int):
        genidx = np.expand_dims(np.expand_dims(np.nonzero(tric['gen'][i])[0],axis=0),axis=2)
        wakidx = np.expand_dims(np.expand_dims(np.nonzero(tric['waktu'][i])[0],axis=1),axis=1)
        konidx = np.expand_dims(np.expand_dims(np.nonzero(tric['kondisi'][i])[0],axis=0),axis=0)
    else:
        genidx = np.expand_dims(np.expand_dims(np.nonzero(i[:54675])[0],axis=0),axis=2)
        wakidx = np.expand_dims(np.expand_dims(np.nonzero(i[54678:])[0],axis=1),axis=1)
        konidx = np.expand_dims(np.expand_dims(np.nonzero(i[54675:54678])[0],axis=0),axis=0)
        
    subarr = D[wakidx, genidx, konidx]
    n_gene = subarr.shape[1]
    n_kondisi = subarr.shape[2]
    n_waktu = subarr.shape[0]
    
# hitung m_iJK (gene)
    m_iJK = np.nanmean(np.nanmean(subarr,axis=2),axis=0)
    m_iJK = np.expand_dims(np.expand_dims(m_iJK,axis=0),axis=2)
        
# hitung m_IjK (kondisi)
    m_IjK = np.nanmean(np.nanmean(subarr,axis=0),axis=0)
    m_IjK = np.expand_dims(np.expand_dims(m_IjK,axis=0),axis=1)
        
# hitung m_IJk (waktu)
    m_IJk = np.nanmean(np.nanmean(subarr,axis=2),axis=1)
    m_IJk = np.expand_dims(np.expand_dims(m_IJk,axis=1),axis=2)
        
# hitung m_IJK
    m_IJK = np.mean(subarr)
        
# hitung MSR
    residue = subarr - m_iJK - m_IjK - m_IJk + (2*m_IJK)
    SR = np.square(residue)
    MSR = np.mean(SR)
    return MSR

# Simulasi 11 delta trimax 2e-05 crossover 4

In [18]:
a = DeltaTrimax(D)
gg11, kk11, ww11, msr11, tqi11 = a.fit(0.00002, 1.2, mask_mode="random", n_triclusters=40)

mask mode: random 

Tricluster  1
 multiple deletion 3 - single deletion 0- node addition 0
------------- MSR:  1.4675325078971638e-05
------------- TQI:  6.694825404176766e-11
G X S X T: 18267  x  3  x  4
Tricluster  2
 multiple deletion 15 - single deletion 0- node addition 6
------------- MSR:  1.2141364278196127e-05
------------- TQI:  1.3784473522021033e-10
G X S X T: 7340  x  3  x  4
Tricluster  3
 multiple deletion 9 - single deletion 0- node addition 14
------------- MSR:  1.3997392941550749e-05
------------- TQI:  1.3941071014651558e-10
G X S X T: 8367  x  3  x  4
Tricluster  4
 multiple deletion 34 - single deletion 0- node addition 7
------------- MSR:  1.1166957194430324e-05
------------- TQI:  1.159023248477428e-10
G X S X T: 8029  x  3  x  4
Tricluster  5
 multiple deletion 30 - single deletion 0- node addition 9
------------- MSR:  1.1376070003434733e-05
------------- TQI:  1.3310949643633264e-10
G X S X T: 7122  x  3  x  4
Tricluster  6
 multiple deletion 58 - single de

In [19]:
#Saving the delta trimax to csv
array11 = [] 
for i in range (len(gg11)):
    x = list(gg11[i]),list(kk11[i]),list(ww11[i]),(tqi11[i])
    array11.append(x)
df11 = pd.DataFrame (array11)
df11.columns = ['gen', 'kondisi', 'waktu','tqi']

In [20]:
df11.to_csv("output11.csv",index= False) 

In [51]:
tric_awal= df11.iloc[:,0:3]

In [52]:
b = Firefly(tric_awal,D)
firefly11 = b.fit(0.00002,4,100)

Waktu Komputasi:  15.261957101027171


In [53]:
msr_total11=[]
tqi11_fa= []
vol_11=[]
for i in range(len(firefly11)):
    print('Tricluster',(i+1))
    s = hitung_delta(firefly11,i)
    gen = np.sum(firefly11['gen'][i])
    kondisi = np.sum(firefly11['kondisi'][i])
    waktu = np.sum(firefly11['waktu'][i])
    vol = gen*kondisi*waktu
    msr_total11.append(s)
    vol_11.append(vol)
    tqi11_fa.append(s/vol)
    print('MSR: ',s)
    print('Volume: ',vol)
    print('TQI: ',str(s/vol))
    print('G X S X T: ', str(gen),'x',str(kondisi),'x',str(waktu))
    print('------------')

Tricluster 1
MSR:  1.4675325078971638e-05
Volume:  219204
TQI:  6.694825404176766e-11
G X S X T:  18267 x 3 x 4
------------
Tricluster 2
MSR:  1.4675325078971638e-05
Volume:  219204
TQI:  6.694825404176766e-11
G X S X T:  18267 x 3 x 4
------------
Tricluster 3
MSR:  1.4675325078971638e-05
Volume:  219204
TQI:  6.694825404176766e-11
G X S X T:  18267 x 3 x 4
------------
Tricluster 4
MSR:  1.4675325078971638e-05
Volume:  219204
TQI:  6.694825404176766e-11
G X S X T:  18267 x 3 x 4
------------
Tricluster 5
MSR:  1.4675325078971638e-05
Volume:  219204
TQI:  6.694825404176766e-11
G X S X T:  18267 x 3 x 4
------------
Tricluster 6
MSR:  1.4675325078971638e-05
Volume:  219204
TQI:  6.694825404176766e-11
G X S X T:  18267 x 3 x 4
------------
Tricluster 7
MSR:  1.4675325078971638e-05
Volume:  219204
TQI:  6.694825404176766e-11
G X S X T:  18267 x 3 x 4
------------
Tricluster 8
MSR:  1.5218994835646314e-05
Volume:  220284
TQI:  6.908806284453848e-11
G X S X T:  18357 x 3 x 4
------------


In [54]:
np.mean(tqi11_fa)

6.870733000734638e-11

In [55]:
np.mean(tqi11)

1.760157724276668e-10

In [182]:
np.mean(msr_total11)

1.5122472263324398e-05

In [187]:
(219204+220824)/2

220014.0

In [56]:
firefly11.to_csv('output_firefly11.csv',index = False)

# Simulasi 12 delta trimax 2e-05 crossover 6

In [57]:
df12 = df11.copy()

In [58]:
tric_awal= df12.iloc[:,0:3]

In [59]:
b = Firefly(tric_awal,D)
firefly12 = b.fit(0.00002,6,100)

Waktu Komputasi:  9.262674963474273


In [60]:
msr_total12=[]
tqi12_fa= []
vol_12=[]
for i in range(len(firefly12)):
    print('Tricluster',(i+1))
    s = hitung_delta(firefly12,i)
    gen = np.sum(firefly12['gen'][i])
    kondisi = np.sum(firefly12['kondisi'][i])
    waktu = np.sum(firefly12['waktu'][i])
    vol = gen*kondisi*waktu
    msr_total12.append(s)
    vol_12.append(vol)
    tqi12_fa.append(s/vol)
    print('MSR: ',s)
    print('Volume: ',vol)
    print('TQI: ',str(s/vol))
    print('G X S X T: ', str(gen),'x',str(kondisi),'x',str(waktu))
    print('------------')

Tricluster 1
MSR:  1.4675325078971638e-05
Volume:  219204
TQI:  6.694825404176766e-11
G X S X T:  18267 x 3 x 4
------------
Tricluster 2
MSR:  1.4675325078971638e-05
Volume:  219204
TQI:  6.694825404176766e-11
G X S X T:  18267 x 3 x 4
------------
Tricluster 3
MSR:  1.4675325078971638e-05
Volume:  219204
TQI:  6.694825404176766e-11
G X S X T:  18267 x 3 x 4
------------
Tricluster 4
MSR:  1.4675325078971638e-05
Volume:  219204
TQI:  6.694825404176766e-11
G X S X T:  18267 x 3 x 4
------------
Tricluster 5
MSR:  1.4675325078971638e-05
Volume:  219204
TQI:  6.694825404176766e-11
G X S X T:  18267 x 3 x 4
------------
Tricluster 6
MSR:  1.4675325078971638e-05
Volume:  219204
TQI:  6.694825404176766e-11
G X S X T:  18267 x 3 x 4
------------
Tricluster 7
MSR:  1.4675325078971638e-05
Volume:  219204
TQI:  6.694825404176766e-11
G X S X T:  18267 x 3 x 4
------------
Tricluster 8
MSR:  1.5021554228243132e-05
Volume:  219744
TQI:  6.835933735730273e-11
G X S X T:  18312 x 3 x 4
------------


In [61]:
np.mean(tqi12_fa)

6.811239777708409e-11

In [183]:
np.mean(msr_total12)

1.496096412712062e-05

In [186]:
(219744+219204)/2

219474.0

In [63]:
firefly12.to_csv('output_firefly12.csv',index = False)

# Simulasi 13 delta trimax 2e-05 crossover 8

In [64]:
df13 = df11.copy()

In [65]:
tric_awal= df13.iloc[:,0:3]

In [66]:
b = Firefly(tric_awal,D)
firefly13 = b.fit(0.00002,8,100)

Waktu Komputasi:  8.88525253534317


In [67]:
msr_total13=[]
tqi13_fa= []
vol_13=[]
for i in range(len(firefly13)):
    print('Tricluster',(i+1))
    s = hitung_delta(firefly13,i)
    gen = np.sum(firefly13['gen'][i])
    kondisi = np.sum(firefly13['kondisi'][i])
    waktu = np.sum(firefly13['waktu'][i])
    vol = gen*kondisi*waktu
    msr_total13.append(s)
    vol_13.append(vol)
    tqi13_fa.append(s/vol)
    print('MSR: ',s)
    print('Volume: ',vol)
    print('TQI: ',str(s/vol))
    print('G X S X T: ', str(gen),'x',str(kondisi),'x',str(waktu))
    print('------------')

Tricluster 1
MSR:  1.4675325078971638e-05
Volume:  219204
TQI:  6.694825404176766e-11
G X S X T:  18267 x 3 x 4
------------
Tricluster 2
MSR:  1.4675325078971638e-05
Volume:  219204
TQI:  6.694825404176766e-11
G X S X T:  18267 x 3 x 4
------------
Tricluster 3
MSR:  1.4675325078971638e-05
Volume:  219204
TQI:  6.694825404176766e-11
G X S X T:  18267 x 3 x 4
------------
Tricluster 4
MSR:  1.4675325078971638e-05
Volume:  219204
TQI:  6.694825404176766e-11
G X S X T:  18267 x 3 x 4
------------
Tricluster 5
MSR:  1.4675325078971638e-05
Volume:  219204
TQI:  6.694825404176766e-11
G X S X T:  18267 x 3 x 4
------------
Tricluster 6
MSR:  1.4675325078971638e-05
Volume:  219204
TQI:  6.694825404176766e-11
G X S X T:  18267 x 3 x 4
------------
Tricluster 7
MSR:  1.4675325078971638e-05
Volume:  219204
TQI:  6.694825404176766e-11
G X S X T:  18267 x 3 x 4
------------
Tricluster 8
MSR:  1.4675325078971638e-05
Volume:  219204
TQI:  6.694825404176766e-11
G X S X T:  18267 x 3 x 4
------------


In [68]:
np.mean(tqi13_fa)

6.694852768433972e-11

In [184]:
np.mean(msr_total13)

1.4675525657135205e-05

In [185]:
(219204+219216)/2

219210.0

In [70]:
firefly13.to_csv('output_firefly13.csv',index = False)

# Simulasi 21 delta trimax 3e-05 crossover 4

In [71]:
a = DeltaTrimax(D)
gg21, kk21, ww21, msr21, tqi21 = a.fit(0.00003, 1.2, mask_mode="random", n_triclusters=40)

mask mode: random 

Tricluster  1
 multiple deletion 2 - single deletion 0- node addition 0
------------- MSR:  2.09454290313267e-05
------------- TQI:  6.127619516507723e-11
G X S X T: 28485  x  3  x  4
Tricluster  2
 multiple deletion 17 - single deletion 0- node addition 6
------------- MSR:  1.752396370570199e-05
------------- TQI:  9.49931899309503e-11
G X S X T: 15373  x  3  x  4
Tricluster  3
 multiple deletion 8 - single deletion 0- node addition 54
------------- MSR:  1.807586917645679e-05
------------- TQI:  1.944896618943059e-10
G X S X T: 7745  x  3  x  4
Tricluster  4
 multiple deletion 25 - single deletion 0- node addition 10
------------- MSR:  1.4088937066397376e-05
------------- TQI:  1.1520734852972701e-10
G X S X T: 10191  x  3  x  4
Tricluster  5
 multiple deletion 22 - single deletion 0- node addition 17
------------- MSR:  1.3942328573624514e-05
------------- TQI:  1.404910174690096e-10
G X S X T: 8270  x  3  x  4
Tricluster  6
 multiple deletion 25 - single delet

In [72]:
#Saving the delta trimax to csv
array21 = [] 
for i in range (len(gg21)):
    x = list(gg21[i]),list(kk21[i]),list(ww21[i]),(tqi21[i])
    array21.append(x)
df21 = pd.DataFrame (array21)
df21.columns = ['gen', 'kondisi', 'waktu','tqi']

In [73]:
df21.to_csv("output21.csv",index= False) 

In [84]:
tric_awal= df21.iloc[:,0:3]

In [85]:
b = Firefly(tric_awal,D)
firefly21 = b.fit(0.00003,4,100)

Waktu Komputasi:  10.19489638408025


In [86]:
msr_total21=[]
tqi21_fa= []
vol_21=[]
for i in range(len(firefly21)):
    print('Tricluster',(i+1))
    s = hitung_delta(firefly21,i)
    gen = np.sum(firefly21['gen'][i])
    kondisi = np.sum(firefly21['kondisi'][i])
    waktu = np.sum(firefly21['waktu'][i])
    vol = gen*kondisi*waktu
    msr_total21.append(s)
    vol_21.append(vol)
    tqi21_fa.append(s/vol)
    print('MSR: ',s)
    print('Volume: ',vol)
    print('TQI: ',str(s/vol))
    print('G X S X T: ', str(gen),'x',str(kondisi),'x',str(waktu))
    print('------------')

Tricluster 1
MSR:  2.09454290313267e-05
Volume:  341820
TQI:  6.127619516507723e-11
G X S X T:  28485 x 3 x 4
------------
Tricluster 2
MSR:  2.09454290313267e-05
Volume:  341820
TQI:  6.127619516507723e-11
G X S X T:  28485 x 3 x 4
------------
Tricluster 3
MSR:  2.09454290313267e-05
Volume:  341820
TQI:  6.127619516507723e-11
G X S X T:  28485 x 3 x 4
------------
Tricluster 4
MSR:  2.09454290313267e-05
Volume:  341820
TQI:  6.127619516507723e-11
G X S X T:  28485 x 3 x 4
------------
Tricluster 5
MSR:  2.09454290313267e-05
Volume:  341820
TQI:  6.127619516507723e-11
G X S X T:  28485 x 3 x 4
------------
Tricluster 6
MSR:  2.09454290313267e-05
Volume:  341820
TQI:  6.127619516507723e-11
G X S X T:  28485 x 3 x 4
------------
Tricluster 7
MSR:  2.09454290313267e-05
Volume:  341820
TQI:  6.127619516507723e-11
G X S X T:  28485 x 3 x 4
------------
Tricluster 8
MSR:  2.09454290313267e-05
Volume:  341820
TQI:  6.127619516507723e-11
G X S X T:  28485 x 3 x 4
------------
Tricluster 9
MSR

In [87]:
np.mean(tqi21_fa)

6.127619516507723e-11

In [88]:
np.mean(tqi21)

1.8492466000597405e-10

In [89]:
firefly21.to_csv('output_firefly21.csv',index = False)

# Simulasi 22 delta trimax 3e-05 crossover 6

In [90]:
df22 = df21.copy()

In [91]:
tric_awal= df22.iloc[:,0:3]

In [92]:
b = Firefly(tric_awal,D)
firefly22 = b.fit(0.00003,6,100)

Waktu Komputasi:  8.677407697836559


In [93]:
msr_total22=[]
tqi22_fa= []
vol_22=[]
for i in range(len(firefly22)):
    print('Tricluster',(i+1))
    s = hitung_delta(firefly22,i)
    gen = np.sum(firefly22['gen'][i])
    kondisi = np.sum(firefly22['kondisi'][i])
    waktu = np.sum(firefly22['waktu'][i])
    vol = gen*kondisi*waktu
    msr_total22.append(s)
    vol_22.append(vol)
    tqi22_fa.append(s/vol)
    print('MSR: ',s)
    print('Volume: ',vol)
    print('TQI: ',str(s/vol))
    print('G X S X T: ', str(gen),'x',str(kondisi),'x',str(waktu))
    print('------------')

Tricluster 1
MSR:  2.09454290313267e-05
Volume:  341820
TQI:  6.127619516507723e-11
G X S X T:  28485 x 3 x 4
------------
Tricluster 2
MSR:  2.09454290313267e-05
Volume:  341820
TQI:  6.127619516507723e-11
G X S X T:  28485 x 3 x 4
------------
Tricluster 3
MSR:  2.09454290313267e-05
Volume:  341820
TQI:  6.127619516507723e-11
G X S X T:  28485 x 3 x 4
------------
Tricluster 4
MSR:  2.09454290313267e-05
Volume:  341820
TQI:  6.127619516507723e-11
G X S X T:  28485 x 3 x 4
------------
Tricluster 5
MSR:  2.09454290313267e-05
Volume:  341820
TQI:  6.127619516507723e-11
G X S X T:  28485 x 3 x 4
------------
Tricluster 6
MSR:  2.09454290313267e-05
Volume:  341820
TQI:  6.127619516507723e-11
G X S X T:  28485 x 3 x 4
------------
Tricluster 7
MSR:  2.09454290313267e-05
Volume:  341820
TQI:  6.127619516507723e-11
G X S X T:  28485 x 3 x 4
------------
Tricluster 8
MSR:  2.09454290313267e-05
Volume:  341820
TQI:  6.127619516507723e-11
G X S X T:  28485 x 3 x 4
------------
Tricluster 9
MSR

In [94]:
np.mean(tqi22_fa)

6.127619516507723e-11

In [95]:
np.mean(df22['tqi'])

1.8492466000597407e-10

In [96]:
firefly22.to_csv('output_firefly22.csv',index = False)

# Simulasi 23 delta trimax 3e-05 crossover 6

In [109]:
df23 = df21.copy() 

In [110]:
tric_awal= df23.iloc[:,0:3]

In [111]:
b = Firefly(tric_awal,D)
firefly23 = b.fit(0.00003,8,100)

Waktu Komputasi:  8.620334057013194


In [112]:
msr_total23=[]
tqi23_fa= []
vol_23=[]
for i in range(len(firefly23)):
    print('Tricluster',(i+1))
    s = hitung_delta(firefly23,i)
    gen = np.sum(firefly23['gen'][i])
    kondisi = np.sum(firefly23['kondisi'][i])
    waktu = np.sum(firefly23['waktu'][i])
    vol = gen*kondisi*waktu
    msr_total23.append(s)
    vol_23.append(vol)
    tqi23_fa.append(s/vol)
    print('MSR: ',s)
    print('Volume: ',vol)
    print('TQI: ',str(s/vol))
    print('G X S X T: ', str(gen),'x',str(kondisi),'x',str(waktu))
    print('------------')

Tricluster 1
MSR:  2.09454290313267e-05
Volume:  341820
TQI:  6.127619516507723e-11
G X S X T:  28485 x 3 x 4
------------
Tricluster 2
MSR:  2.09454290313267e-05
Volume:  341820
TQI:  6.127619516507723e-11
G X S X T:  28485 x 3 x 4
------------
Tricluster 3
MSR:  2.09454290313267e-05
Volume:  341820
TQI:  6.127619516507723e-11
G X S X T:  28485 x 3 x 4
------------
Tricluster 4
MSR:  2.09454290313267e-05
Volume:  341820
TQI:  6.127619516507723e-11
G X S X T:  28485 x 3 x 4
------------
Tricluster 5
MSR:  2.09454290313267e-05
Volume:  341820
TQI:  6.127619516507723e-11
G X S X T:  28485 x 3 x 4
------------
Tricluster 6
MSR:  2.09454290313267e-05
Volume:  341820
TQI:  6.127619516507723e-11
G X S X T:  28485 x 3 x 4
------------
Tricluster 7
MSR:  2.09454290313267e-05
Volume:  341820
TQI:  6.127619516507723e-11
G X S X T:  28485 x 3 x 4
------------
Tricluster 8
MSR:  2.09454290313267e-05
Volume:  341820
TQI:  6.127619516507723e-11
G X S X T:  28485 x 3 x 4
------------
Tricluster 9
MSR

In [113]:
np.mean(tqi23_fa)

6.127619516507723e-11

In [114]:
firefly23.to_csv('output_firefly23.csv',index = False)

# Simulasi 31 delta trimax 4e-05 crossover 4

In [115]:
a = DeltaTrimax(D)
gg31, kk31, ww31, msr31, tqi31 = a.fit(0.00004, 1.2, mask_mode="random", n_triclusters=40)

mask mode: random 

Tricluster  1
 multiple deletion 1 - single deletion 0- node addition 0
------------- MSR:  3.330214677199463e-05
------------- TQI:  6.460966399707166e-11
G X S X T: 42953  x  3  x  4
Tricluster  2
 multiple deletion 9 - single deletion 0- node addition 14
------------- MSR:  2.3206150124878333e-05
------------- TQI:  7.355822912665885e-10
G X S X T: 2629  x  3  x  4
Tricluster  3
 multiple deletion 9 - single deletion 0- node addition 23
------------- MSR:  1.929188359234362e-05
------------- TQI:  1.465102493418969e-10
G X S X T: 10973  x  3  x  4
Tricluster  4
 multiple deletion 14 - single deletion 0- node addition 57
------------- MSR:  2.2597717760871877e-05
------------- TQI:  1.3128438000134712e-10
G X S X T: 14344  x  3  x  4
Tricluster  5
 multiple deletion 16 - single deletion 0- node addition 31
------------- MSR:  1.824197176263088e-05
------------- TQI:  1.4396858732385391e-10
G X S X T: 10559  x  3  x  4
Tricluster  6
 multiple deletion 21 - single d

In [116]:
#Saving the delta trimax to csv
array31 = [] 
for i in range (len(gg31)):
    x = list(gg31[i]),list(kk31[i]),list(ww31[i]),(tqi31[i])
    array31.append(x)
df31 = pd.DataFrame (array31)
df31.columns = ['gen', 'kondisi', 'waktu','tqi']

In [117]:
df31.to_csv("output31.csv",index= False) 

In [118]:
tric_awal= df31.iloc[:,0:3]

In [119]:
b = Firefly(tric_awal,D)
firefly31 = b.fit(0.00004,4,100)

Waktu Komputasi:  9.035296070575715


In [120]:
msr_total31=[]
tqi31_fa= []
vol_31=[]
for i in range(len(firefly31)):
    print('Tricluster',(i+1))
    s = hitung_delta(firefly31,i)
    gen = np.sum(firefly31['gen'][i])
    kondisi = np.sum(firefly31['kondisi'][i])
    waktu = np.sum(firefly31['waktu'][i])
    vol = gen*kondisi*waktu
    msr_total31.append(s)
    vol_31.append(vol)
    tqi31_fa.append(s/vol)
    print('MSR: ',s)
    print('Volume: ',vol)
    print('TQI: ',str(s/vol))
    print('G X S X T: ', str(gen),'x',str(kondisi),'x',str(waktu))
    print('------------')

Tricluster 1
MSR:  3.330214677199463e-05
Volume:  515436
TQI:  6.460966399707166e-11
G X S X T:  42953 x 3 x 4
------------
Tricluster 2
MSR:  3.330214677199463e-05
Volume:  515436
TQI:  6.460966399707166e-11
G X S X T:  42953 x 3 x 4
------------
Tricluster 3
MSR:  3.330214677199463e-05
Volume:  515436
TQI:  6.460966399707166e-11
G X S X T:  42953 x 3 x 4
------------
Tricluster 4
MSR:  3.330214677199463e-05
Volume:  515436
TQI:  6.460966399707166e-11
G X S X T:  42953 x 3 x 4
------------
Tricluster 5
MSR:  3.330214677199463e-05
Volume:  515436
TQI:  6.460966399707166e-11
G X S X T:  42953 x 3 x 4
------------
Tricluster 6
MSR:  3.330214677199463e-05
Volume:  515436
TQI:  6.460966399707166e-11
G X S X T:  42953 x 3 x 4
------------
Tricluster 7
MSR:  3.330214677199463e-05
Volume:  515436
TQI:  6.460966399707166e-11
G X S X T:  42953 x 3 x 4
------------
Tricluster 8
MSR:  3.330214677199463e-05
Volume:  515436
TQI:  6.460966399707166e-11
G X S X T:  42953 x 3 x 4
------------
Triclust

In [121]:
np.mean(tqi31_fa)

6.460966399707166e-11

In [122]:
np.mean(tqi31)

2.580149019272917e-10

In [123]:
firefly31.to_csv('output_firefly31.csv',index = False)

# Simulasi 32 delta trimax 4e-05 crossover 4

In [124]:
df32 = df31.copy()

In [125]:
tric_awal= df32.iloc[:,0:3]

In [126]:
b = Firefly(tric_awal,D)
firefly32 = b.fit(0.00004,6,100)

Waktu Komputasi:  8.712194792429607


In [127]:
msr_total32=[]
tqi32_fa= []
vol_32=[]
for i in range(len(firefly32)):
    print('Tricluster',(i+1))
    s = hitung_delta(firefly32,i)
    gen = np.sum(firefly32['gen'][i])
    kondisi = np.sum(firefly32['kondisi'][i])
    waktu = np.sum(firefly32['waktu'][i])
    vol = gen*kondisi*waktu
    msr_total32.append(s)
    vol_32.append(vol)
    tqi32_fa.append(s/vol)
    print('MSR: ',s)
    print('Volume: ',vol)
    print('TQI: ',str(s/vol))
    print('G X S X T: ', str(gen),'x',str(kondisi),'x',str(waktu))
    print('------------')

Tricluster 1
MSR:  3.330214677199463e-05
Volume:  515436
TQI:  6.460966399707166e-11
G X S X T:  42953 x 3 x 4
------------
Tricluster 2
MSR:  3.330214677199463e-05
Volume:  515436
TQI:  6.460966399707166e-11
G X S X T:  42953 x 3 x 4
------------
Tricluster 3
MSR:  3.330214677199463e-05
Volume:  515436
TQI:  6.460966399707166e-11
G X S X T:  42953 x 3 x 4
------------
Tricluster 4
MSR:  3.330214677199463e-05
Volume:  515436
TQI:  6.460966399707166e-11
G X S X T:  42953 x 3 x 4
------------
Tricluster 5
MSR:  3.330214677199463e-05
Volume:  515436
TQI:  6.460966399707166e-11
G X S X T:  42953 x 3 x 4
------------
Tricluster 6
MSR:  3.330214677199463e-05
Volume:  515436
TQI:  6.460966399707166e-11
G X S X T:  42953 x 3 x 4
------------
Tricluster 7
MSR:  3.330214677199463e-05
Volume:  515436
TQI:  6.460966399707166e-11
G X S X T:  42953 x 3 x 4
------------
Tricluster 8
MSR:  3.330214677199463e-05
Volume:  515436
TQI:  6.460966399707166e-11
G X S X T:  42953 x 3 x 4
------------
Triclust

In [128]:
np.mean(tqi32_fa)

6.460966399707166e-11

In [130]:
firefly32.to_csv('output_firefly32.csv',index = False)

# Simulasi 33 delta trimax 4e-05 crossover 4

In [131]:
df33 = df31.copy()

In [132]:
tric_awal= df33.iloc[:,0:3]

In [133]:
b = Firefly(tric_awal,D)
firefly33 = b.fit(0.00004,8,100)

Waktu Komputasi:  8.590740485986073


In [134]:
msr_total33=[]
tqi33_fa= []
vol_33=[]
for i in range(len(firefly33)):
    print('Tricluster',(i+1))
    s = hitung_delta(firefly33,i)
    gen = np.sum(firefly33['gen'][i])
    kondisi = np.sum(firefly33['kondisi'][i])
    waktu = np.sum(firefly33['waktu'][i])
    vol = gen*kondisi*waktu
    msr_total33.append(s)
    vol_33.append(vol)
    tqi33_fa.append(s/vol)
    print('MSR: ',s)
    print('Volume: ',vol)
    print('TQI: ',str(s/vol))
    print('G X S X T: ', str(gen),'x',str(kondisi),'x',str(waktu))
    print('------------')

Tricluster 1
MSR:  3.330214677199463e-05
Volume:  515436
TQI:  6.460966399707166e-11
G X S X T:  42953 x 3 x 4
------------
Tricluster 2
MSR:  3.330214677199463e-05
Volume:  515436
TQI:  6.460966399707166e-11
G X S X T:  42953 x 3 x 4
------------
Tricluster 3
MSR:  3.330214677199463e-05
Volume:  515436
TQI:  6.460966399707166e-11
G X S X T:  42953 x 3 x 4
------------
Tricluster 4
MSR:  3.330214677199463e-05
Volume:  515436
TQI:  6.460966399707166e-11
G X S X T:  42953 x 3 x 4
------------
Tricluster 5
MSR:  3.330214677199463e-05
Volume:  515436
TQI:  6.460966399707166e-11
G X S X T:  42953 x 3 x 4
------------
Tricluster 6
MSR:  3.330214677199463e-05
Volume:  515436
TQI:  6.460966399707166e-11
G X S X T:  42953 x 3 x 4
------------
Tricluster 7
MSR:  3.330214677199463e-05
Volume:  515436
TQI:  6.460966399707166e-11
G X S X T:  42953 x 3 x 4
------------
Tricluster 8
MSR:  3.330214677199463e-05
Volume:  515436
TQI:  6.460966399707166e-11
G X S X T:  42953 x 3 x 4
------------
Triclust

In [135]:
np.mean(tqi33_fa)

6.460966399707166e-11

In [86]:
np.mean(tqi33)

2.6992102346958895e-10

In [87]:
firefly33.to_csv('output_firefly33.csv',index = False)

# Simulasi 41 delta trimax 5e-05 crossover 4

In [136]:
a = DeltaTrimax(D)
gg41, kk41, ww41, msr41, tqi41 = a.fit(0.00005, 1.2, mask_mode="random", n_triclusters=40)

mask mode: random 

Tricluster  1
 multiple deletion 1 - single deletion 0- node addition 0
------------- MSR:  3.330214677199463e-05
------------- TQI:  6.460966399707166e-11
G X S X T: 42953  x  3  x  4
Tricluster  2
 multiple deletion 8 - single deletion 0- node addition 39
------------- MSR:  2.7792926416953294e-05
------------- TQI:  1.2244011426373306e-10
G X S X T: 18916  x  3  x  4
Tricluster  3
 multiple deletion 8 - single deletion 0- node addition 19
------------- MSR:  2.0586687388423118e-05
------------- TQI:  1.278453895497871e-10
G X S X T: 13419  x  3  x  4
Tricluster  4
 multiple deletion 22 - single deletion 1315- node addition 1
------------- MSR:  2.3244413638141686e-05
------------- TQI:  7.569054060313544e-11
G X S X T: 34122  x  3  x  3
Tricluster  5
 multiple deletion 12 - single deletion 0- node addition 35
------------- MSR:  3.1668915918784e-05
------------- TQI:  7.805608774224589e-10
G X S X T: 3381  x  3  x  4
Tricluster  6
 multiple deletion 32 - single d

In [137]:
#Saving the delta trimax to csv
array41 = [] 
for i in range (len(gg41)):
    x = list(gg41[i]),list(kk41[i]),list(ww41[i]),(tqi41[i])
    array41.append(x)
df41= pd.DataFrame (array41)
df41.columns = ['gen', 'kondisi', 'waktu','tqi']

In [138]:
df41.to_csv("output41.csv",index=False) 

In [139]:
tric_awal= df41.iloc[:,0:3]

In [140]:
b = Firefly(tric_awal,D)
firefly41 = b.fit(0.00005,4,100)

Waktu Komputasi:  9.470394734541575


In [141]:
msr_total41 =[]
tqi41_fa= []
vol_41=[]
for i in range(len(firefly41)):
    print('Tricluster',(i+1))
    s = hitung_delta(firefly41,i)
    gen = np.sum(firefly41['gen'][i])
    kondisi = np.sum(firefly41['kondisi'][i])
    waktu = np.sum(firefly41['waktu'][i])
    vol = gen*kondisi*waktu
    msr_total41.append(s)
    vol_41.append(vol)
    tqi41_fa.append(s/vol)
    print('MSR: ',s)
    print('Volume: ',vol)
    print('TQI: ',str(s/vol))
    print('G X S X T: ', str(gen),'x',str(kondisi),'x',str(waktu))
    print('------------')

Tricluster 1
MSR:  3.330214677199463e-05
Volume:  515436
TQI:  6.460966399707166e-11
G X S X T:  42953 x 3 x 4
------------
Tricluster 2
MSR:  3.330214677199463e-05
Volume:  515436
TQI:  6.460966399707166e-11
G X S X T:  42953 x 3 x 4
------------
Tricluster 3
MSR:  3.330214677199463e-05
Volume:  515436
TQI:  6.460966399707166e-11
G X S X T:  42953 x 3 x 4
------------
Tricluster 4
MSR:  3.330214677199463e-05
Volume:  515436
TQI:  6.460966399707166e-11
G X S X T:  42953 x 3 x 4
------------
Tricluster 5
MSR:  3.330214677199463e-05
Volume:  515436
TQI:  6.460966399707166e-11
G X S X T:  42953 x 3 x 4
------------
Tricluster 6
MSR:  3.330214677199463e-05
Volume:  515436
TQI:  6.460966399707166e-11
G X S X T:  42953 x 3 x 4
------------
Tricluster 7
MSR:  3.330214677199463e-05
Volume:  515436
TQI:  6.460966399707166e-11
G X S X T:  42953 x 3 x 4
------------
Tricluster 8
MSR:  3.330214677199463e-05
Volume:  515436
TQI:  6.460966399707166e-11
G X S X T:  42953 x 3 x 4
------------
Triclust

In [142]:
np.mean(tqi41)

2.7846509146612166e-10

In [143]:
np.mean(tqi41_fa)

6.460966399707166e-11

In [144]:
firefly41.to_csv('output_firefly41.csv',index= False)

# Simulasi 42 delta trimax 5e-05 crossover 4

In [145]:
df42 = df41.copy()

In [146]:
tric_awal= df42.iloc[:,0:3]

In [147]:
b = Firefly(tric_awal,D)
firefly42 = b.fit(0.00005,6,100)

Waktu Komputasi:  9.036970448493957


In [148]:
msr_total42 =[]
tqi42_fa= []
vol_42=[]
for i in range(len(firefly42)):
    print('Tricluster',(i+1))
    s = hitung_delta(firefly42,i)
    gen = np.sum(firefly42['gen'][i])
    kondisi = np.sum(firefly42['kondisi'][i])
    waktu = np.sum(firefly42['waktu'][i])
    vol = gen*kondisi*waktu
    msr_total42.append(s)
    vol_42.append(vol)
    tqi42_fa.append(s/vol)
    print('MSR: ',s)
    print('Volume: ',vol)
    print('TQI: ',str(s/vol))
    print('G X S X T: ', str(gen),'x',str(kondisi),'x',str(waktu))
    print('------------')

Tricluster 1
MSR:  3.330214677199463e-05
Volume:  515436
TQI:  6.460966399707166e-11
G X S X T:  42953 x 3 x 4
------------
Tricluster 2
MSR:  3.330214677199463e-05
Volume:  515436
TQI:  6.460966399707166e-11
G X S X T:  42953 x 3 x 4
------------
Tricluster 3
MSR:  3.330214677199463e-05
Volume:  515436
TQI:  6.460966399707166e-11
G X S X T:  42953 x 3 x 4
------------
Tricluster 4
MSR:  3.330214677199463e-05
Volume:  515436
TQI:  6.460966399707166e-11
G X S X T:  42953 x 3 x 4
------------
Tricluster 5
MSR:  3.330214677199463e-05
Volume:  515436
TQI:  6.460966399707166e-11
G X S X T:  42953 x 3 x 4
------------
Tricluster 6
MSR:  3.330214677199463e-05
Volume:  515436
TQI:  6.460966399707166e-11
G X S X T:  42953 x 3 x 4
------------
Tricluster 7
MSR:  3.330214677199463e-05
Volume:  515436
TQI:  6.460966399707166e-11
G X S X T:  42953 x 3 x 4
------------
Tricluster 8
MSR:  3.330214677199463e-05
Volume:  515436
TQI:  6.460966399707166e-11
G X S X T:  42953 x 3 x 4
------------
Triclust

In [150]:
np.mean(tqi42_fa)

6.460966399707166e-11

In [151]:
firefly42.to_csv('output_firefly42.csv',index= False)

# Simulasi 43 delta trimax 5e-05 crossover 4

In [152]:
df43 = df41.copy()

In [153]:
tric_awal= df43.iloc[:,0:3]

In [154]:
b = Firefly(tric_awal,D)
firefly43 = b.fit(0.00005,8,100)

Waktu Komputasi:  8.944281351566314


In [155]:
msr_total43 =[]
tqi43_fa= []
vol_43=[]
for i in range(len(firefly43)):
    print('Tricluster',(i+1))
    s = hitung_delta(firefly43,i)
    gen = np.sum(firefly43['gen'][i])
    kondisi = np.sum(firefly43['kondisi'][i])
    waktu = np.sum(firefly43['waktu'][i])
    vol = gen*kondisi*waktu
    msr_total43.append(s)
    vol_43.append(vol)
    tqi43_fa.append(s/vol)
    print('MSR: ',s)
    print('Volume: ',vol)
    print('TQI: ',str(s/vol))
    print('G X S X T: ', str(gen),'x',str(kondisi),'x',str(waktu))
    print('------------')

Tricluster 1
MSR:  3.330214677199463e-05
Volume:  515436
TQI:  6.460966399707166e-11
G X S X T:  42953 x 3 x 4
------------
Tricluster 2
MSR:  3.330214677199463e-05
Volume:  515436
TQI:  6.460966399707166e-11
G X S X T:  42953 x 3 x 4
------------
Tricluster 3
MSR:  3.330214677199463e-05
Volume:  515436
TQI:  6.460966399707166e-11
G X S X T:  42953 x 3 x 4
------------
Tricluster 4
MSR:  3.330214677199463e-05
Volume:  515436
TQI:  6.460966399707166e-11
G X S X T:  42953 x 3 x 4
------------
Tricluster 5
MSR:  3.330214677199463e-05
Volume:  515436
TQI:  6.460966399707166e-11
G X S X T:  42953 x 3 x 4
------------
Tricluster 6
MSR:  3.330214677199463e-05
Volume:  515436
TQI:  6.460966399707166e-11
G X S X T:  42953 x 3 x 4
------------
Tricluster 7
MSR:  3.330214677199463e-05
Volume:  515436
TQI:  6.460966399707166e-11
G X S X T:  42953 x 3 x 4
------------
Tricluster 8
MSR:  3.330214677199463e-05
Volume:  515436
TQI:  6.460966399707166e-11
G X S X T:  42953 x 3 x 4
------------
Triclust

In [157]:
np.mean(tqi43_fa)

6.460966399707166e-11

In [158]:
firefly43.to_csv('output_firefly43.csv',index= False)

# Simulasi 51 delta trimax 6e-05 crossover 4

In [159]:
a = DeltaTrimax(D)
gg51, kk51, ww51, msr51, tqi51 = a.fit(0.00006, 1.2, mask_mode="random", n_triclusters=40)

mask mode: random 

Tricluster  1
 multiple deletion 1 - single deletion 0- node addition 0
------------- MSR:  3.330214677199463e-05
------------- TQI:  6.460966399707166e-11
G X S X T: 42953  x  3  x  4
Tricluster  2
 multiple deletion 8 - single deletion 0- node addition 39
------------- MSR:  2.7808196272167286e-05
------------- TQI:  1.2246853870348135e-10
G X S X T: 18922  x  3  x  4
Tricluster  3
 multiple deletion 8 - single deletion 0- node addition 18
------------- MSR:  2.0600056671713254e-05
------------- TQI:  1.2777606172753538e-10
G X S X T: 13435  x  3  x  4
Tricluster  4
 multiple deletion 24 - single deletion 873- node addition 1
------------- MSR:  2.6621830545597555e-05
------------- TQI:  7.638822332291046e-11
G X S X T: 38723  x  3  x  3
Tricluster  5
 multiple deletion 10 - single deletion 0- node addition 32
------------- MSR:  3.5235100859989384e-05
------------- TQI:  7.42980365637428e-10
G X S X T: 3952  x  3  x  4
Tricluster  6
 multiple deletion 48 - single

In [160]:
#Saving the delta trimax to csv
array51 = [] 
for i in range (len(gg51)):
    x = list(gg51[i]),list(kk51[i]),list(ww51[i]),(tqi51[i])
    array51.append(x)
df51= pd.DataFrame (array51)
df51.columns = ['gen', 'kondisi', 'waktu','tqi']

In [161]:
df51.to_csv("output51.csv",index=False) 

In [162]:
tric_awal= df51.iloc[:,0:3]

In [163]:
b = Firefly(tric_awal,D)
firefly51 = b.fit(0.00006,4,100)

Waktu Komputasi:  8.678583908081055


In [164]:
msr_total51 =[]
tqi51_fa= []
vol_51=[]
for i in range(len(firefly51)):
    print('Tricluster',(i+1))
    s = hitung_delta(firefly51,i)
    gen = np.sum(firefly51['gen'][i])
    kondisi = np.sum(firefly51['kondisi'][i])
    waktu = np.sum(firefly51['waktu'][i])
    vol = gen*kondisi*waktu
    msr_total51.append(s)
    vol_51.append(vol)
    tqi51_fa.append(s/vol)
    print('MSR: ',s)
    print('Volume: ',vol)
    print('TQI: ',str(s/vol))
    print('G X S X T: ', str(gen),'x',str(kondisi),'x',str(waktu))
    print('------------')

Tricluster 1
MSR:  3.330214677199463e-05
Volume:  515436
TQI:  6.460966399707166e-11
G X S X T:  42953 x 3 x 4
------------
Tricluster 2
MSR:  3.330214677199463e-05
Volume:  515436
TQI:  6.460966399707166e-11
G X S X T:  42953 x 3 x 4
------------
Tricluster 3
MSR:  3.330214677199463e-05
Volume:  515436
TQI:  6.460966399707166e-11
G X S X T:  42953 x 3 x 4
------------
Tricluster 4
MSR:  3.330214677199463e-05
Volume:  515436
TQI:  6.460966399707166e-11
G X S X T:  42953 x 3 x 4
------------
Tricluster 5
MSR:  3.330214677199463e-05
Volume:  515436
TQI:  6.460966399707166e-11
G X S X T:  42953 x 3 x 4
------------
Tricluster 6
MSR:  3.330214677199463e-05
Volume:  515436
TQI:  6.460966399707166e-11
G X S X T:  42953 x 3 x 4
------------
Tricluster 7
MSR:  3.330214677199463e-05
Volume:  515436
TQI:  6.460966399707166e-11
G X S X T:  42953 x 3 x 4
------------
Tricluster 8
MSR:  3.330214677199463e-05
Volume:  515436
TQI:  6.460966399707166e-11
G X S X T:  42953 x 3 x 4
------------
Triclust

In [165]:
np.mean(tqi51)

2.5822406519290454e-10

In [166]:
np.mean(tqi51_fa)

6.460966399707166e-11

In [167]:
firefly51.to_csv('output_firefly51.csv',index= False)

# Simulasi 52 delta trimax 6e-05 crossover 6

In [168]:
df52 = df51.copy()

In [169]:
tric_awal= df52.iloc[:,0:3]

In [170]:
b = Firefly(tric_awal,D)
firefly52 = b.fit(0.00006,6,100)

Waktu Komputasi:  8.654472243785857


In [171]:
msr_total52 =[]
tqi52_fa= []
vol_52=[]
for i in range(len(firefly52)):
    print('Tricluster',(i+1))
    s = hitung_delta(firefly52,i)
    gen = np.sum(firefly52['gen'][i])
    kondisi = np.sum(firefly52['kondisi'][i])
    waktu = np.sum(firefly52['waktu'][i])
    vol = gen*kondisi*waktu
    msr_total52.append(s)
    vol_52.append(vol)
    tqi52_fa.append(s/vol)
    print('MSR: ',s)
    print('Volume: ',vol)
    print('TQI: ',str(s/vol))
    print('G X S X T: ', str(gen),'x',str(kondisi),'x',str(waktu))
    print('------------')

Tricluster 1
MSR:  3.330214677199463e-05
Volume:  515436
TQI:  6.460966399707166e-11
G X S X T:  42953 x 3 x 4
------------
Tricluster 2
MSR:  3.330214677199463e-05
Volume:  515436
TQI:  6.460966399707166e-11
G X S X T:  42953 x 3 x 4
------------
Tricluster 3
MSR:  3.330214677199463e-05
Volume:  515436
TQI:  6.460966399707166e-11
G X S X T:  42953 x 3 x 4
------------
Tricluster 4
MSR:  3.330214677199463e-05
Volume:  515436
TQI:  6.460966399707166e-11
G X S X T:  42953 x 3 x 4
------------
Tricluster 5
MSR:  3.330214677199463e-05
Volume:  515436
TQI:  6.460966399707166e-11
G X S X T:  42953 x 3 x 4
------------
Tricluster 6
MSR:  3.330214677199463e-05
Volume:  515436
TQI:  6.460966399707166e-11
G X S X T:  42953 x 3 x 4
------------
Tricluster 7
MSR:  3.330214677199463e-05
Volume:  515436
TQI:  6.460966399707166e-11
G X S X T:  42953 x 3 x 4
------------
Tricluster 8
MSR:  3.330214677199463e-05
Volume:  515436
TQI:  6.460966399707166e-11
G X S X T:  42953 x 3 x 4
------------
Triclust

In [173]:
np.mean(tqi52_fa)

6.460966399707166e-11

In [174]:
firefly52.to_csv('output_firefly52.csv',index= False)

# Simulasi 53 delta trimax 6e-05 crossover 6

In [175]:
df53 = df51.copy()

In [176]:
tric_awal= df53.iloc[:,0:3]

In [177]:
b = Firefly(tric_awal,D)
firefly53 = b.fit(0.00006,8,100)

Waktu Komputasi:  6.9818103035291035


In [178]:
msr_total53 =[]
tqi53_fa= []
vol_53=[]
for i in range(len(firefly53)):
    print('Tricluster',(i+1))
    s = hitung_delta(firefly53,i)
    gen = np.sum(firefly53['gen'][i])
    kondisi = np.sum(firefly53['kondisi'][i])
    waktu = np.sum(firefly53['waktu'][i])
    vol = gen*kondisi*waktu
    msr_total53.append(s)
    vol_53.append(vol)
    tqi53_fa.append(s/vol)
    print('MSR: ',s)
    print('Volume: ',vol)
    print('TQI: ',str(s/vol))
    print('G X S X T: ', str(gen),'x',str(kondisi),'x',str(waktu))
    print('------------')

Tricluster 1
MSR:  3.330214677199463e-05
Volume:  515436
TQI:  6.460966399707166e-11
G X S X T:  42953 x 3 x 4
------------
Tricluster 2
MSR:  3.330214677199463e-05
Volume:  515436
TQI:  6.460966399707166e-11
G X S X T:  42953 x 3 x 4
------------
Tricluster 3
MSR:  3.330214677199463e-05
Volume:  515436
TQI:  6.460966399707166e-11
G X S X T:  42953 x 3 x 4
------------
Tricluster 4
MSR:  3.330214677199463e-05
Volume:  515436
TQI:  6.460966399707166e-11
G X S X T:  42953 x 3 x 4
------------
Tricluster 5
MSR:  3.330214677199463e-05
Volume:  515436
TQI:  6.460966399707166e-11
G X S X T:  42953 x 3 x 4
------------
Tricluster 6
MSR:  3.330214677199463e-05
Volume:  515436
TQI:  6.460966399707166e-11
G X S X T:  42953 x 3 x 4
------------
Tricluster 7
MSR:  3.330214677199463e-05
Volume:  515436
TQI:  6.460966399707166e-11
G X S X T:  42953 x 3 x 4
------------
Tricluster 8
MSR:  3.330214677199463e-05
Volume:  515436
TQI:  6.460966399707166e-11
G X S X T:  42953 x 3 x 4
------------
Triclust

In [180]:
np.mean(tqi53_fa)

6.460966399707166e-11

In [181]:
firefly53.to_csv('output_firefly53.csv',index= False)

# ---------------